# Lab 6 - Coder Agent

A coder agent doesn't necessarily mean that it generates code - but more broadly that it can write code and run it in order to solve a task.

In [1]:
from agents import Agent, Runner, trace, function_tool
import docker
import tempfile
import os
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [11]:
#client = docker.from_env()
client = docker.DockerClient(base_url='unix:///Users/joshuajanzen/.colima/default/docker.sock')
image = "python:3.12-slim"

In [12]:
client.containers.run(image="python:3.12-slim", command=["python", "-c", "print(2+2)"], remove=True)

b'4\n'

In [13]:
@function_tool
def execute_python(code: str) -> str:
    """
    Execute the given Python code inside a Docker container with python:3.12-slim,
    and return whatever is printed to stdout.
    You must print the result of the code to stdout in order to retrieve it.
    This uses the python:3.12-slim image and so it does not have scientific libraries installed;
    write simple python 3.12 code using the standard library only. Do not use numpy or scipy.
    IMPORTANT: You must print the result of the code in order to retrieve it.

    Args:
        code: The Python code to run. Remember to print the result.

    """
    print(f"Executing code: {code}")
    with tempfile.TemporaryDirectory() as tmpdir:
        script_path = os.path.join(tmpdir, "script.py")
        volumes = {tmpdir: {"bind": "/tmp", "mode": "ro"}}
        command = ["python", "/tmp/script.py"]
        with open(script_path, "w") as f:
            f.write(code)
        logs = client.containers.run(image=image, command=command, volumes=volumes, remove=True)
    result = logs.decode("utf-8")
    print(f"Result: {result}")
    return result

In [16]:
#execute_python.description
execute_python.params_json_schema

{'properties': {'code': {'description': 'The Python code to run. Remember to print the result.',
   'title': 'Code',
   'type': 'string'}},
 'required': ['code'],
 'title': 'execute_python_args',
 'type': 'object',
 'additionalProperties': False}

In [17]:
instructions = """
You are a Coder Agent that writes python, uses a tool to execute the code, and returns the results,
in order to solve the user's problem.
"""

In [18]:
number = 5 * 11 * 47 * 307
input = f"What are the prime factors of {number}? Reply only with the answer."

In [21]:
agent = Agent("Coder Agent", model="gpt-4.1", instructions=instructions, tools=[execute_python])

with trace("Coder Agent"):
    result = await Runner.run(agent, input)
    print(result.final_output)

Executing code: def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors

print(prime_factors(793595))
Executing code: def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors

print(prime_factors(793595))
Executing code: def prime_factors(n):
    factors = []
    d = 2
    while d * d <= n:
        while n % d == 0:
            factors.append(d)
            n //= d
        d += 1
    if n > 1:
        factors.append(n)
    print(factors)

prime_factors(793595)
There appears to be a temporary issue running code. Would you like the algorithm so you can run it locally, or would you like me to try again?


In [22]:


input = """
Write a python function to calculate the price of an option using the Black-Scholes model.
Then use your tool to execute the code and calculate the price of this option:
GOOGL
Stock Price: 150
Strike Price: 155
Time to maturity: 3 years
Risk free rate: 1%
Volatility: 20%
Dividend Yield: 0%
Respond with the calculated price of the option only.
"""

In [23]:

agent = Agent("Coder Agent", model="gpt-4.1-mini", instructions=instructions, tools=[execute_python])

with trace("Coder Agent"):
    result = await Runner.run(agent, input)
    print(result.final_output)

Executing code: import math

from math import log, sqrt, exp
from math import erf

def black_scholes_price(S, K, T, r, sigma, q=0, option_type='call'):
    # Cumulative distribution function for standard normal distribution
    def N(x):
        return 0.5 * (1 + erf(x / sqrt(2)))

    d1 = (log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)

    if option_type == 'call':
        price = S * exp(-q * T) * N(d1) - K * exp(-r * T) * N(d2)
    else:
        price = K * exp(-r * T) * N(-d2) - S * exp(-q * T) * N(-d1)

    return price

# Given option parameters
S = 150     # Stock price
K = 155     # Strike price
T = 3       # Time to maturity in years
r = 0.01    # Risk free interest rate
sigma = 0.20 # Volatility
q = 0.0     # Dividend yield

price = black_scholes_price(S, K, T, r, sigma, q, 'call')
print(price)
Executing code: import math
from math import log, sqrt, exp, erf

def black_scholes_price(S, K, T, r, sigma, q=0, option_type='call'):